In [ ]:
#!/usr/bin/env python
# coding: utf-8

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from datetime import datetime
from selenium.webdriver.support.ui import Select
import pandas as pd
from sqlalchemy import create_engine
import logging
import time

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def init_webdriver():
    options = webdriver.FirefoxOptions()
    #options.add_argument("-headless")
    return webdriver.Firefox(options=options)

def accept_cookies(driver):
    cookies_xpath = "//*[@id='onetrust-accept-btn-handler']"
    try:
        cookies_popup = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.XPATH, cookies_xpath)))
        cookies_popup.click()
    except Exception as e:
        logging.warning("Cookies popup not found or not clickable: %s", str(e))

def select_refresh_interval(driver):
    try:
        dropdown_element = driver.find_element(By.CSS_SELECTOR, '.\[\&_\.bn-select-field\]\:mobile\:g-\[border-none\,h-auto\,w-auto\,p-0\] > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)')
        time.sleep(3)
        dropdown_element.click()
        element_5s = driver.find_element(By.CSS_SELECTOR, 'div.bn-select-overlay-options:nth-child(1) > div:nth-child(3)')
        element_5s.click()
    except Exception as e:
        logging.error("Error selecting refresh interval: %s", str(e))

def extract_data(driver, formatted_time, adv_list_name="AdvTableList", data_type="buy"):
    adv_list = driver.find_element(By.CLASS_NAME, adv_list_name)
    rows = adv_list.find_element(By.TAG_NAME, 'table').find_elements(By.TAG_NAME, 'tr')
    count_of_rows = len(rows)

    name_xpath = f"/html/body/div[3]/div/div[2]/main/div[2]/div[3]/div/div[1]/div/div/div/table/tbody/tr[{{}}]/td[1]/div/div[1]"
    price_xpath = f"/html/body/div[3]/div/div[2]/main/div[2]/div[3]/div/div[1]/div/div/div/table/tbody/tr[{{}}]/td[2]/div/div[1]"
    usdt_xpath = f"/html/body/div[3]/div/div[2]/main/div[2]/div[3]/div/div[1]/div/div/div/table/tbody/tr[{{}}]/td[3]/div/div[1]"
    min_mmk_xpath = f"/html/body/div[3]/div/div[2]/main/div[2]/div[3]/div/div[1]/div/div/div/table/tbody/tr[{{}}]/td[3]/div/div[2]/div[1]"
    max_mmk_xpath = f"/html/body/div[3]/div/div[2]/main/div[2]/div[3]/div/div[1]/div/div/div/table/tbody/tr[{{}}]/td[3]/div/div[2]/div[3]"

    names_list, prices_list, usdt_list, min_mmk_list, max_mmk_list, timelog_list = [], [], [], [], [], [formatted_time] * (count_of_rows - 1)

    for i in range(1, count_of_rows - 1):
        try:
            name = adv_list.find_element(By.XPATH, name_xpath.format(i)).text.split('\n')[1]
        except:
            name = "Unknown"
        names_list.append(name)
        
        price = float(adv_list.find_element(By.XPATH, price_xpath.format(i)).text.replace(',', ''))
        prices_list.append(price)
        
        usdt = float(adv_list.find_element(By.XPATH, usdt_xpath.format(i)).text.replace(' USDT', '').replace(',', ''))
        usdt_list.append(usdt)
        
        min_mmk = float(adv_list.find_element(By.XPATH, min_mmk_xpath.format(i)).text.replace('$', '').replace(',', ''))
        min_mmk_list.append(min_mmk)
        
        max_mmk = float(adv_list.find_element(By.XPATH, max_mmk_xpath.format(i)).text.replace('$', '').replace(',', ''))
        max_mmk_list.append(max_mmk)

    data = list(zip(timelog_list, names_list, prices_list, usdt_list, min_mmk_list, max_mmk_list))
    columns = ['DateTime', f'{data_type.capitalize()} Name', f'{data_type.capitalize()} Price', f'{data_type.capitalize()} USDT', f'{data_type.capitalize()} Min MMK', f'{data_type.capitalize()} Max MMK']
    df = pd.DataFrame(data, columns=columns)

    best_price = df.iloc[0][f'{data_type.capitalize()} Price']
    df[f'Best {data_type.capitalize()} Price'] = best_price

    best_vol = df.iloc[0][f'{data_type.capitalize()} Max MMK']
    df[f'Best {data_type.capitalize()} Volume'] = best_vol

    top4 = df.head(4)
    top4_avg_price = sum(top4[f'{data_type.capitalize()} Price']) / len(top4[f'{data_type.capitalize()} Price'])
    df[f'Top 4 Avg {data_type.capitalize()} Price'] = top4_avg_price

    top4_total_vol = sum(top4[f'{data_type.capitalize()} Max MMK'])
    df[f'Top 4 Total {data_type.capitalize()} Volume'] = top4_total_vol

    return df

def switch_tab(driver, tab_index):
    try:
        button = driver.find_element(By.CSS_SELECTOR, f"div.bn-tab__segment-outline:nth-child({tab_index})")
        button.click()
        time.sleep(5)
    except Exception as e:
        logging.error("Error switching tab: %s", str(e))

def insert_dataframe_to_postgres(df, db_config):
    db_uri = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}"
    try:
        engine = create_engine(db_uri)
        df.to_sql('exchange_data', engine, index=False, if_exists='append', method='multi', schema='exchange_data')
        logging.info("Data inserted successfully.")
    except Exception as e:
        logging.error("Error inserting data: %s", str(e))

def main():
    driver = init_webdriver()
    driver.get('https://p2p.binance.com/en/trade/all-payments/USDT?fiat=USD')
    
    accept_cookies(driver)
    select_refresh_interval(driver)

    db_config = {
        'host': 'yourhost',
        'database': 'yourdatabase',
        'user': 'youruser',
        'password': 'yourpassword'
    }

    while True:
        try:
            current_time = datetime.now()
            formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
            
            buy_df = extract_data(driver, formatted_time, data_type="buy")
            logging.info("Buy data extracted.")

            switch_tab(driver, 2)
            sell_df = extract_data(driver, formatted_time, data_type="sell")
            logging.info("Sell data extracted.")

            out_df = buy_df.join(sell_df.set_index('DateTime'), on='DateTime')
            insert_dataframe_to_postgres(out_df, db_config)
            
            switch_tab(driver, 1)
            time.sleep(3600)
            logging.info("Data insertion successful at %s", formatted_time)
        except Exception as e:
            logging.error("Error occurred: %s", str(e))
            time.sleep(5)

if __name__ == "__main__":
    main()
